<font color = red>Introduction to Business Analytics:<br>Using Python for Better Business Decisions <br><br><font color = blue>Solutions Manual</font>
=======
<br>
    <center><img src="http://dataanalyticscorp.com/wp-content/uploads/2018/03/logo.png"></center>
<br>
Taught by: 

* Walter R. Paczkowski, Ph.D. 

    * My Affliations: [Data Analytics Corp.](http://www.dataanalyticscorp.com/) and [Rutgers University](https://economics.rutgers.edu/people/teaching-personnel)
    * [Email Me With Questions](mailto:walt@dataanalyticscorp.com)
    * [Learn About Me](http://www.dataanalyticscorp.com/)
    * [See My LinkedIn Profile](https://www.linkedin.com/in/walter-paczkowski-a17a1511/)
 

###  <font color = black> Reset the Data from Lesson 1 </font>

Resetting the data will ensure that your solutions will match the work you did in Lesson 1.

In [ ]:
##
## Load packages
##
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
##
## For modeling, notice the new import command for
## the formula API and the summary option
##
import statsmodels.api as sm
import statsmodels.formula.api as smf
##
## Import the data.  The parse_dates argument says to 
## treat Tdate as a date object.
##
file = r'../Data/furniture/final data files/orders.csv'
df_orders = pd.read_csv( file, parse_dates = [ 'Tdate' ] )
pd.set_option('display.max_columns', 8)
##
## Initial Calculations
##
x = [ 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc' ]
df_orders[ 'Tdisc' ] = df_orders[ x ].sum( axis = 1 )
##
df_orders[ 'Pprice' ] = df_orders.Lprice*( 1 - df_orders.Tdisc )
##
df_orders[ 'Rev' ] = df_orders.Usales * df_orders.Pprice
##
df_orders[ 'Con' ] = df_orders.Rev - df_orders.Mcost
df_orders[ 'CM' ] = df_orders.Con/df_orders.Rev
##
df_orders[ 'netRev' ] = ( df_orders.Usales - df_orders.returnAmount )*df_orders.Pprice
df_orders[ 'lostRev' ] = df_orders.Rev - df_orders.netRev
##
##
## Import a second DataFrame on the customers
##
file = r'../Data/furniture/final data files/customers.csv'
df_cust = pd.read_csv( file )
##
## Do an inner join using CID as the link
##
df = pd.merge( df_orders, df_cust, on = 'CID' )
#################################################
##      The following is for Lesson 3: OLS
#################################################
##
## Import train_test_split
##
from sklearn.model_selection import train_test_split
##
## Create the X and y data for splitting
##
y = df[ 'Usales' ]
x = [ 'Pprice', 'Ddisc', 'Odisc', 'Cdisc', 'Pdisc', 'Region', 'buyerRating' ]
X = df[ x ]
##
## Split the data.  The default is 3/4 train.
##
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.25,
                                                    random_state = 42 ) 
## 
## Merge the X and y training data for 
## model training.
##
## Rename the y variable: Usales
##
yy = pd.DataFrame( { 'Usales':y_train } )
train = yy.merge( X_train, left_index = True, right_index = True )
## 
## Merge the X and y testing data sets for predicting
##
## Rename the y variable Usales
##
yy = pd.DataFrame( { 'Usales':y_test } )
test = yy.merge( X_test, left_index = True, right_index = True )
##
## Add log Usales and log Pprice to the training data
## The log is based on the Numpy function log1p
## Note: log1p( x ) = log( 1 + x )
##
train[ 'log_Usales' ] = np.log1p( train.Usales )
train[ 'log_Pprice' ] = np.log1p( train.Pprice )
##
## Repeat for the testing data
##
test[ 'log_Usales' ] = np.log1p( test.Usales )
test[ 'log_Pprice' ] = np.log1p( test.Pprice )

## Lesson 1 Exercise Solutions

### Solution \#1.1

The *Pocket Price* is the list price less total discounts or total leakages.  It is the amount the business "pockets" and is the amount the customer actually pays.  The pocket price formula is $Pprice = Lprice \times (1  - Tdisc)$.  Calculate the pocket price and display the first five records for the list price and pocket price.

In [ ]:
df_orders[ 'Pprice' ] = df_orders.Lprice*( 1 - df_orders.Tdisc )
x = [ 'Lprice', 'Tdisc', 'Pprice' ]
df_orders[ x ].head()

### Solution \#1.2

Merge your order DataFrame and the customer DataFrame.  Name the merged DataFrame *df*.

In [ ]:
##
## Do an inner join using CID as the link
##
df = pd.merge( df_orders, df_cust, on = 'CID' )
df.head()

### Solution \#1.3

Using your merged orders/customers DataFrame, *df*, create a summary statistics display.  What is the skewness of the Total Discount (Tdisc)?

In [ ]:
df.describe().T

Total Discount is slightly left skewed since the distance between Q! and the median is bigger than the distance between Q3 qnd the median.

## Appendix 1 Exercises Solutions

### Solution \#A 1.3.1

Calculate total revenue as $Rev = Usales \times Pprice$.  Use the df_orders DataFrame.

In [ ]:
##
## Multiply Unit Sales and Pocket Price
##
df_orders[ 'Rev' ] = df_orders.Usales * df_orders.Pprice
##
## Create a list of unit sales, pocket price, and revenue
##
x = [ 'Usales', 'Pprice', 'Rev' ]
df_orders[ x ].head()

### Solution \#A 1.3.2

*Contribution* and *contribution margin* are two values financial analysts often examine.  Contribution is comparable to what economists call *profit* but is more restricted in that it just refers to a product without considering any fixed or overhead costs.  Contribution is $Con = Revenue - Material~Cost$ and contribution margin is $CM = \dfrac{Con}{Revenue}$.  Calculate both quantities and display the first 5 records of unit sales, pocket price, material cost, revenue, contribution, and contribution margin.  Use the df_orders DataFrame.

In [ ]:
##
## Contribution: Subtract Material Cost (Mcost from the Data Dictionary) from Revenue
##
df_orders[ 'Con' ] = df_orders.Rev - df_orders.Mcost
##
## Contribution Margin: Divide Contribution by Revenue
##
df_orders[ 'CM' ] = df_orders.Con/df_orders.Rev
##
## Create a list to display
##
x = [ 'Usales', 'Pprice', 'Mcost', 'Rev', 'Con', 'CM' ]
df_orders[ x ].head( )

### Solution \#A 1.3.3

Some products are returned so another revenue number, *revenue net of returns*, is more meaningful and revealing for business decisions.  Net revenue is
<br><br>
$Net Revenue = (Unit Sales - Returns) \times Pocket Price$.
<br><br>
Calculate net revenue and call it 'netRev'.  Also calculate the loss in revenue due to the returns and call it 'lostRev'.  Display the first five records of the DataFrame using just gross revenue, net revenue, and the lost revenue due to returns.  Use the *df_orders* DataFrame.  

In [ ]:
##
## Net Revenue: Subtract the amount returned (returnAmount from the Data Dictionary) and 
## multiply by the Pocket Price
##
df_orders[ 'netRev' ] = ( df_orders.Usales - df_orders.returnAmount )*df_orders.Pprice
##
## Lost Revenue: Total revenue less the net revenue
##
df_orders[ 'lostRev' ] = df_orders.Rev - df_orders.netRev
##
## Create a list to display
##
x = [ 'Rev', 'netRev', 'lostRev' ]
df_orders[ x ].head()

### Solution \#A 1.3.4

Display the descriptive statistics for gross revenue, net revenue, contribution, and contribution margin.  Round the answers to two decimal places.  Use the *df_orders* DataFrame. 

In [ ]:
##
## Create a list to display
##
x = [ 'Rev', 'netRev', 'Con', 'CM' ]
round( df_orders[ x ].describe().T, 2)

### Solution \#A 1.4.1

Create a correlation matrix and corresponding heatmap for gross revenue, net revenue, contribution, and contribution margin.  What would you tell your product manager about the correlations?  Use the *df_orders* DataFrame.  

In [ ]:
##
## Create a list to display
##
x = [ 'Rev', 'netRev', 'Con', 'CM' ]
##
## Calculate the correlations
##
cor = df_orders[ x ].corr()
cor

In [ ]:
##
## Create a heatmap
##
sns.heatmap( cor ).set_title( 'Heatmap of the Correlation Matrix' )

**_Interpretation_**

The correlation matrix shows a high positive linear association between revenue and net revenue, contribution, and contribution margin.  This should be expected since the last three are all ultimately a function of revenue.

## Lesson 2 Exercise Solutions

### Solution \#2.1 

Check the Customer Loyalty and Buyer Rating counts and proportions.

In [ ]:
df.loyaltyProgram.value_counts()  ## Use the "normalize = True" argument for proportions 

In [ ]:
df.buyerRating.value_counts()  ## Use the "normalize = True" argument for proportions

### Solution  \#2.2 

Examine the Midwestern region. This is more complicated since there are missing values in the Midwest. First use df.dropna( axis = 0, inplace = True ) to remove them.

Hint: Use the Pandas *dropna* method with the *inplace = True* argument.

In [ ]:
##
## Example
##
## Drop all rows with at least one missing value
## This example uses a temporary DataFrame
##
x = [ 'Tdisc', 'Ddisc', 'Cdisc', 'Odisc', 'Pdisc' ]
tmp = df.loc[ df.Region == 'Midwest', x ]
##
## Before
##
print( '\nBefore:\n' )
print( tmp.info() )
##
## After
##
tmp.dropna( inplace = True )  ## axis = 0 is the default 
print( '\nAfter:\n' )
print( tmp.info() )

In [ ]:
x = [ 'Ddisc', 'Cdisc', 'Odisc', 'Pdisc' ]
df_midwest = df.loc[ df.Region == 'Midwest', x ]
df_midwest.dropna( inplace = True )
##
## Box plot of discounts
##
ax = sns.boxplot(x = "variable", y = "value", data = pd.melt( df_midwest ) )
ax.set( title = 'Discount Distribution\nMidwest Marketing Region', 
       xlabel = 'Type of Discount',
      ylabel = 'Discount Amount')
ax.set_xticklabels( [ 'Dealer', 'Order\nSize', 'Competitive', 'Pickup' ] )

### Solution \#2.3

Examine the distribution of net revenue by region, loyalty program, and buyer rating using boxplots.  What can you conclude?

In [ ]:
##
## Net revenue by region
##
ax = sns.boxplot(x = "Region", y = "netRev", data = df )
ax.set( title = 'Net Revenue Distribution\nMarketing Region', 
       xlabel = 'Marketing Region',
      ylabel = 'Net Revenue')

In [ ]:
##
## Net revenue by Loyalty Program
##
ax = sns.boxplot(x = "loyaltyProgram", y = "netRev", data = df )
ax.set( title = 'Net Revenue Distribution\nLoyalty Program Membership', 
       xlabel = 'Loyalty Program Membership',
      ylabel = 'Net Revenue')

In [ ]:
##
## Net revenue by Buyer Rating
##
ax = sns.boxplot(x = "buyerRating", y = "netRev", data = df )
ax.set( title = 'Net Revenue Distribution\nBuyer Rating', 
       xlabel = 'Buyer Rating',
      ylabel = 'Net Revenue')

**_Interpretation_**

The distributions are difficult to interpret because there are so many outliers.  You could make them more interpretatble by deleting outliers.  Try the following for the Buyer Rating distribution.

In [ ]:
##
## Net revenue by Buyer Rating with net Revenue less than $300
##
tmp = df[ df.netRev < 300 ]
ax = sns.boxplot(x = "buyerRating", y = "netRev", data = tmp )
ax.set( title = 'Net Revenue Distribution\nBuyer Rating\nNet Revenue Less than $300', 
       xlabel = 'Buyer Rating',
      ylabel = 'Net Revenue')

**_Interpretation_**

The distribution is the same regardless of the buyer rating for net revenue below \\$300.  You cn examine the distribution for net revenue greater than \\$300.

In [ ]:
##
## Net revenue by Buyer Rating with net Revenue greater than $300
##
tmp = df[ df.netRev > 300 ]
ax = sns.boxplot(x = "buyerRating", y = "netRev", data = tmp )
ax.set( title = 'Net Revenue Distribution\nBuyer Rating\nNet Revenue Greater than $300', 
       xlabel = 'Buyer Rating',
      ylabel = 'Net Revenue')

### Solution \#2.4

Study the relationship between any two variables of your choice. What can you conclude?

In [ ]:
##
## No solution -- you can look at two variables of your choice.
##

## Lesson 3

### Solution \#3.1

Estimate a new OLS model by adding the buyer rating to the above model. Interpret your results.  Is the buyer rating important for sales?

Hint: Buyer rating is categorical so you have to create dummies for the rating.  

In [ ]:
## 
## OLS
##
## There are four steps for estimatng a model:
##   1. define a formula (i.e., the specific model to estimate)
##   2. instantiate the model (i.e., specify it)
##   3. fit the model
##   4. summarize the fitted model
##
## ===> Step 1: Define a formula
##
## The formula uses a “~” to separate the left-hand side from the right-hand side
## of a model and a “+” to add columns to the right-hand side.  A “-” sign (not 
## used here) can be used to remove columns from the right-hand side (e.g.,
## remove or omit the constant term which is always included by default). 
##
formula = 'log_Usales ~ log_Pprice + Ddisc + Odisc + Cdisc + Pdisc + C( Region ) + C( buyerRating )'
##
## Since Region is categorical, you must create dummies for the regions.  You
## do this using 'C( Region )' to indicate that Region is categorical.
##
## ===> Step 2: Instantiate the OLS model
##
mod = smf.ols( formula, data = train )
##
## ===> Step 3: Fit the instantiated model
##      Recommendation: number your models
##      This numbering includes an "E" for "Exercise"
##
regE01 = mod.fit()
##
## ===> Step 4: Summarize the fitted model
##
print( regE01.summary() )


**_Interpretation_**

The buyer rating is highly insignificant so this variable is not important and can be omitted in a next iteration of estimation.

### Solution \#3.2

Test the Null Hypothesis that all the buyer rating estimated parameters are zero.  That is, there is no difference among the ratings.

In [ ]:
##
## F-test for the buyer ratings
##
hypothesis = ' ( C(buyerRating)[T.Good] = 0, C(buyerRating)[T.Poor] = 0 ) '
##
## Run and print an F-test 
##
f_test = regE01.f_test( hypothesis )
f_test.summary()

**_Interpretation_**

Buyer Rating is highly insignificant (i.e., do not reject the Null Hypothesis) because the p-value is 0.68 which is greater than 0.05.  We already know this result.

### Solution \#3.4

Interpret the decision tree.